In [23]:
#
# Imports
#
import numpy as np

class BackPropagationNetwork:
    """A back-propagation network"""
    #
    # Class members
    #
    layerCount = 0
    shape = None
    weights = []
        
    #
    # Class methods
    #
    def __init__(self, layerSize):
        """Initialize the network"""
            
    # Layer info
        self.layerCount = len(layerSize) - 1
        self.shape = layerSize
            
    # Input/Output data from the last Run
        self._layerInput = []
        self._layerOutput = []
            
        # Create the weight arrays
        for (l1,l2) in zip(layerSize[:-1], layerSize[1:]):
            self.weights.append(np.random.normal(scale=0.1, size = (l2, l1+1)))
    #
    # Run method
    #
    def Run(self, input):
        """Run the network based on the input data"""
        
        lnCases = input.shape[0]
        
        # Clear out the previous intermediate value lists
        self._layerInput = []
        self._layerOutput = []
        
        # Run it
        for index in range(self.layerCount):
            #Determine layer input
            if index == 0:
                layerInput = self.weights[0].dot(np.vstack([input.T, np.ones([1, lnCases])]))
            else:
                layerInput = self.weights[index].dot(np.vstack([self._layerOutput[-1], np.ones([1, lnCases])]))
            
            self._layerInput.append(layerInput)
            self._layerOutput.append(self.sgm(layerInput))
        return self._layerOutput[-1].T
    
    
    #
    # TrainEpoch method
    #
    def TrainEpoch(self, input, target, trainingRate = 0.2):
        """This method trains the network for one epoch"""
        
        delta = []
        lnCases = input.shape[0]
        
        # First run the network
        self.Run(input)
        
        # Calculate our deltas
        for index in reversed(range(self.layerCount)):
            if index == self.layerCount - 1:
                # Compare to the target values
                output_delta = self._layerOutput[index] - target.T
                error = np.sum(output_delta**2)
                delta.append(output_delta * self.sgm(self._layerInput[index], True))
            else:
                # Compare to the following layer's delta
                delta_pullback = self.weights[index + 1].T.dot(delta[-1])
                delta.append(delta_pullback[:-1, :1]* self.sgm(self._layerInput[index], True))
            
        # Compute weight deltas
        for index in range(self.layerCount):
            delta_index = self.layerCount - 1 - index
            
            if index == 0:
                layerOutput = np.vstack([input.T, np.ones([1, lnCases])])
            else:
                layerOutput = np.vstack([self._layerOutput[index - 1], np.ones([1, self._layerOutput[index - 1].shape[1]])])
            
            weightDelta = np.sum(\
                                 layerOutput[None,:,:].transpose(2,0,1)*delta[delta_index][None,:,:].transpose(2,1,0)\
                                 , axis = 0)
            self.weights[index] -= trainingRate*weightDelta  
        return error 
            
        
    # Transfer functions
    def sgm(self, x, Derivative=False):
        if not Derivative:
            return 1.0 / (1.0 + np.exp(-x))
        else:
            out = self.sgm(x)
            return out*(1-out)
                    
#in
# If run as a script, create a test object
#
if __name__ == "__main__":
    bpn = BackPropagationNetwork((2,2,1))
    print(bpn.shape)
    print(bpn.weights)
    
    lvInput = np.array([[0,0], [1,1], [0,1], [1,0]])
    lvTarget = np.array([[0.05], [0.05], [0.95], [0.95]])
    
    lnMax = 100000
    lnErr = 1e-5
    for i in range(lnMax+1):
        err = bpn.TrainEpoch(lvInput, lvTarget)
        if i % 2500 == 0:
            print("Iteration {0}\tError: {1:0.6f}".format(i, err))
        if err <= lnErr:
            print("Minimum error reached at iteration {0}".format(i))
            break
            
    # Display output   
    lvOutput = bpn.Run(lvInput)
    print("Input: {0}\nOutput: {1}".format(lvInput, lvOutput))

(2, 2, 1)
[array([[ 0.0866502 , -0.01198355,  0.06798401],
       [-0.09072377, -0.12075773, -0.0255062 ]]), array([[ 0.01876659,  0.13698191,  0.09589688]])]
Iteration 0	Error: 0.817179
Iteration 2500	Error: 0.809988
Iteration 5000	Error: 0.809931
Iteration 7500	Error: 0.809954
Iteration 10000	Error: 0.810076
Iteration 12500	Error: 0.810019
Iteration 15000	Error: 0.809995
Iteration 17500	Error: 0.809945
Iteration 20000	Error: 0.810024
Iteration 22500	Error: 0.809978
Iteration 25000	Error: 0.810028
Iteration 27500	Error: 0.809988
Iteration 30000	Error: 0.810025
Iteration 32500	Error: 0.809985
Iteration 35000	Error: 0.810005
Iteration 37500	Error: 0.809987
Iteration 40000	Error: 0.810000
Iteration 42500	Error: 0.810010
Iteration 45000	Error: 0.809993
Iteration 47500	Error: 0.809997
Iteration 50000	Error: 0.810001
Iteration 52500	Error: 0.810009
Iteration 55000	Error: 0.809996
Iteration 57500	Error: 0.809992
Iteration 60000	Error: 0.809997
Iteration 62500	Error: 0.810000
Iteration 65000	